In [ ]:
"""
Using callback to act on a model during training

1. When you’re training a model, there are many things you can’t predict from the start. In particular, you can’t tell how many epochs
   will be needed to get to an optimal validation loss. The examples so far have adopted the strategy of training for enough epochs that you
   begin overfitting, using the first run to figure out the proper number of epochs to train for, and then finally launching a new training
   run from scratch using this optimal number. Of course, this approach is wasteful.

2. A much better way to handle this is to stop training when you measure that the validation loss in no longer improving. This can be
   achieved using a Keras callback. A callback is an object (a class instance implementing specific methods) that is passed to the model
   in the call to fit and that is called by the model at various points during training. It has access to all the available data about
   the state of the model and its performance, and it can take action: interrupt training, save a model, load a different weight set, or
   otherwise alter the state of the model.

3. Some examples of ways you can use callbacks:
   (*) Model checkpointing — Saving the current weights of the model at different points during training.
   (*) Early stopping — Interrupting training when the validation loss is no longer improving (and of course, saving the best model
                         obtained during training).
   (*) Dynamically adjusting the value of certain parameters during training—Such as the learning rate of the optimizer.
   (*) Logging training and validation metrics during training, or visualizing the representations learned by the model as they’re updated 
"""

In [ ]:
"""
The model checkpoint & early stopping callbacks

1. You can use the EarlyStopping callback to interrupt training once a target metric being monitored has stopped improving
   for a fixed number of epochs.

2. This callback is typically used in combination with ModelCheckpoint, which lets you continually save the model during training
   and, optionally, save only the current best model so far.
"""

# Interrupts training when accuracy has stopped improving for more than one epoch 
# Saves the current weights after every epoch, and won’t overwrite the model file unless val_loss has improve
callbacks_list = [ keras.callbacks.EarlyStopping(monitor='acc', patience=1,),
                   keras.callbacks.ModelCheckpoint(filepath='my_model.h5', monitor='val_loss', save_best_only=True,)]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

In [ ]:
"""
The reduce LR on plateau callback

You can use this callback to reduce the learning rate when the validation loss has stopped improving. Reducing or increasing the learning
rate in case of a loss plateau is an effective strategy to get out of local minima during training.
"""

# Divides the learning rate by 10 when triggered
# The callback is triggered after the validation loss has stopped improving for 10 epochs
callbacks_list = [ keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,)]

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

In [ ]:
"""
Write your own callback

1. If you need to take a specific action during training that isn’t covered by one of the built-in callbacks, you can write your own
   callback. Callbacks are implemented by subclassing the class keras.callbacks.Callback. You can then implement any number of the following
   transparently named methods, which are called at various points during training:
   -- on_epoch_begin
   -- on_epoch_end
   -- on_batch_begin
   -- on_batch_end
   -- on_train_begin
   -- on_train_end

2. These methods all are called with a logs argument, which is a dictionary containing information about the previous batch, epoch,
   or training run: training and validation metrics, and so on. Additionally, the callback has access to the following attributes:
   (*) self.model — The model instance from which the callback is being called
   (*) self.validation_data — The value of what was passed to fit as validation data
"""
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)

    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()

In [ ]:
"""
TensorBoard

1. To do good research or develop good models, you need rich, frequent feedback about what’s going on inside your models during your
   experiments. That’s the point of running experiments: to get information about how well a model performs—as much information as
   possible. Making progress is an iterative process, or loop: you start with an idea and express it as an experiment, attempting to
   validate or invalidate your idea. You run this experiment and process the information it generates. This inspires your next idea.
   The more iterations of this loop you’re able to run, the more refined and powerful your ideas become. Keras helps you go from idea to
   experiment in the least possible time, and fast GPUs can help you get from experiment to result as quickly as possible.

2.  TensorBoard is a browser-based visualization tool that comes packaged with TensorFlow. The key purpose of TensorBoard is to help you
    visually monitor everything that goes on inside your model during training. If you’re monitoring more information than just the
    model’s final loss, you can develop a clearer vision of what the model does and doesn’t do, and you can make progress more quickly:
    (*) Visually monitoring metrics during training
    (*) Visualizing your model architecture
    (*) Visualizing histograms of activations and gradients
    (*) Exploring embeddings in 3D
"""
# Before you start using TensorBoard, you need to create a directory where you’ll store the log files it generates.
#!mkdir tf_borad_log

In [ ]:
"""
Text-classification model to use with TensorBoard
"""
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import RMSprop
import numpy as np

max_features = 10000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

model = Sequential()

model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu')) # 32: number of filters, 7: kernal size
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])

# Tensorflow callback: Records activation histograms every 1 epoch & embedding data every 1 epoch
callbacks = [ callbacks.TensorBoard(log_dir='./tf_borad_log',
                                    histogram_freq=1,)]
                                    #embeddings_layer_names='embed')] ???

history = model.fit(x_train,
                    y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks,
                    verbose=2)

In [ ]:
from tensorflow.keras.utils  import plot_model

plot_model(model, to_file='model.png')